In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-may-2021/train.csv", index_col="id")
test = pd.read_csv("/kaggle/input/tabular-playground-series-may-2021/test.csv", index_col="id")

In [ ]:
y = train["target"]
train.drop(columns="target", inplace=True)

In [ ]:
df = pd.concat([train,test])

In [ ]:
y = y.str[-1].astype(int) - 1

Before creating the logistic regression model we need to encode this categorical variables and we are going to encode them with one-hot encoding. There are variables with many categories and if we onehot encode all the variables we get 1378 variables as you can see below. So we are going to group the small categories that have less than 1000 cases, then onehot encode and finally train the logistic regression.

In [ ]:
df.nunique().sum()

In [ ]:
def group_small_categories(df,s=1000):
    for i in df.columns:
        val_counts = df[i].value_counts()
        for val in val_counts[val_counts<s].index:
            mask = df[i]==val
            df.loc[mask,i] = 100

In [ ]:
df_onehot = df.copy()
group_small_categories(df_onehot)

We onehot encode with pandas get_dummies function.

In [ ]:
df_onehot = pd.get_dummies(df_onehot.astype(str)).astype(int)
df_onehot.shape

Now we train the logistic regression, predict and submit.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression(solver="newton-cg", multi_class="ovr", C=1, max_iter=300, n_jobs=-1)
lr.fit(df_onehot[:len(train)], y)

In [ ]:
sub = pd.read_csv("/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv", index_col="id")
sub[["Class_1","Class_2","Class_3","Class_4"]] = lr.predict_proba(df_onehot[len(train):])
sub.to_csv("lr_onehot.csv")